

---


# **TCO- Tarea 3: PLANIFICACIÓN DE LA PRODUCCIÓN EN CENTRALES ELÉCTRICAS**

Alexander Olza Rodriguez

---





## **INTRODUCCIÓN: Descripción del problema**


Supongamos que existen tres centrales eléctricas: Una nuclear, una eólica y una térmica. Cada una tiene diferentes costes y beneficios asociados: Hay un coste por poner en marcha o parar cada una de ellas, hay costes por funcionamiento, y distintos beneficios derivados de la producción de energía. Además, cada una de las centrales tiene una cierta capacidad de producción. Entre las tres, deben satisfacer la demanda horaria de un país.

Se pretende planificar el funcionamiento y la producción de cada central en un intervalo de 24 horas. 

Nos encontramos ante un problema de programación lineal entera mixta (MIP): Contiene variables continuas (la producción de cada planta en cada periodo) y binarias (las decisiones de apagar, encender o mantener funcionando cada planta en cada periodo).

### **RESUMEN DE PARÁMETROS:**


---
**VARIABLES:**

$ y_{jk} =\left \{
\begin{array}{ll}
      1 & \textrm{si la central $j$ se enciende al inicio del periodo $k$ } \\
      0 & \textrm{en otro caso} \\
\end{array} 
\right.$ \\

$ z_{jk} =\left \{
\begin{array}{ll}
      1 & \textrm{si la central $j$ se apaga al inicio del periodo $k$ } \\
      0 & \textrm{en otro caso} \\
\end{array} 
\right.$ \\

$ v_{jk} =\left \{
\begin{array}{ll}
      1 & \textrm{si la central $j$ está funcionando en el periodo $k$ } \\
      0 & \textrm{en otro caso} \\
\end{array} 
\right.$ \\

$ p_{jk} \geq 0\quad \textrm{producción de la central $j$ en el periodo $k$}$  \\

**DATOS:**

- $PMax_j$: Capacidad máxima de producción de la central $j$.
- $Pmin_j$: Capacidad mínima de producción de la central $j$.
- $S_j$: Rampa máxima de subida de carga en la central $j$ 

  (Definición: Máximo incremento de potencia permitido entre periodos consecutivos).
- $T_j$: Rampa máxima de bajada de carga en la central $j$

 (Def.: Máximo decremento permitido entre periodos consecutivos).
- $C_j$: Coste de arranque de la central $j$.
- $E_j$: Coste de parada de la central $j$.
- $A_j$: Coste de funcionamiento horario de la central $j$.
- $B_j$: Beneficio por unidad de energía producida en la central $j$.
- $D_k$: Demanda de energía en el periodo $k$.
- $R_k$: Reserva de energía en el periodo $k$.


### **DATOS CONCRETOS: Capacidad de producción, demanda, costes y beneficios**

---



Se ha tomado la demanda horaria programada de la red eléctrica española peninsular, entre las 21:00 del 10-12-2020 y las 21:00 del 11-12-2020. Los datos están en unidades de 1000 MW.

In [1]:
demandas=[0,35.772,32.850,29.316,26.999,25.086,24.319,
          23.627,23.419,24.095,27.184,31.694,33.905,
          35.022,35.481,35.904,34.118,34.982,34.136,
          33.012,32.559,32.811,34.185,34.576,34.696] #25 periodos, k=0,...,24

El resto de los datos han sido arbitrariamente escogidos, siguiendo las consideraciones que se presentan a continuación.

La **central nuclear** tiene una gran capacidad de producción, con $PMax_0=35$ y $Pmin_0=20$, las más altas. El coste de arranque, de parada y de funcionamiento también es elevado: $C_0=20$, $E_0=1$, $A_0=10$. Supongamos que se considera una fuente de energía neutra, para la que no existen ayudas pero tampoco cuotas extra. Por lo tanto, el beneficio obtenido al producir energía nuclear es medio: $B_0=2.3$. La potencia puede aumentar $S_0=25$ unidades o descender $T_0=13$ unidades entre periodos consecutivos.

La **central eólica** tiene baja capacidad de producción en comparación ($Pmin_1=7,PMax_1=20$) y bajo coste de arranque, parada y  funcionamiento ($C_1=5$, $E_1=0.3$, $A_1=4$). Al ser energía renovable, existen ayudas para promocionarla y su beneficio es mayor ($B_1=5.5$). La potencia puede aumentar $S_1=7$ unidades o descender $T_1=15$ unidades entre periodos consecutivos.


La **central térmica** tiene capacidad de producción media-alta ($Pmin_2=13, PMax_2=27$). Sus costes de arranque, parada y funcionamiento son intermedios ($C_2=17,E_2=0.5,A_2=6$). Existen impuestos adicionales por producción contaminante, así que el beneficio es el más bajo ($B_2=1.15$). La potencia puede aumentar $S_2=17$ unidades o descender $T_2=13$ unidades entre periodos consecutivos.





In [2]:
PMax=[35,20,27]
Pmin=[20,7,13]
Subida=[25,7,17]
Bajada=[13,15,13]
A=[10,4,6] #coste fijo funcionamiento
C=[20,5,17] #coste arranque
E=[1.0,0.3,0.5] #coste parada
B=[2.3,5.5,1.15] #beneficio variable (produccion)

### **RESTRICCIONES:** 

---



- Al inicio, todas las centrales están paradas. No hay demanda ni producción.

  $y_{j0}=z_{j0}=v_{j0}=p_{j0}=0\quad \forall j$ 

- Siempre que una central esté funcionando, su potencia (productividad horaria) debe estar entre dos umbrales ($ PMax$ y $Pmin$):

 $ Pmin_jv_{jk}\leq p_{jk}\leq PMax_jv_{jk}\quad \forall j; \forall k$.
-Entre periodos consecutivos, el incremento de producción no debe sobrepasar un límite $S_j$, llamado rampa máxima de subida de carga:

  $p_{jk+1}-p_{jk}\leq S_j\quad \forall j; k=0,...,K-1$

- Análogamente, el decremento de producción ha de ser menor que $T_j$, la rampa máxima de bajada de carga:

  $p_{jk}-p_{jk+1}\leq T_j\quad \forall j; k=0,...,K-1$ 

- Los cambios de estado deben ser coherentes con la variable de funcionamiento. Si la central se enciende en el periodo $k$, estará funcionando en el periodo $k$ y no lo estará en el $k-1$. Si se apaga en el periodo $k$, debe haber funcionado en el $k-1$ y no funcionará en el $k$:

  $y_{jk}-z_{jk}=v_{jk}-v_{jk-1}\quad \forall j; k=1,...,K$

- Entre todas las centrales deben satisfacer la demanda de cada periodo:

  $\sum_j p_{jk}=D_k \quad \forall j,k$

- La potencia máxima total de las centrales en funcionamiento debe superar la demanda, al menos, en una cantidad de reserva $R_k$, que se ha fijado en un 10% de la demanda prevista. Teniendo en cuenta la restricción anterior, esto evita que todas las centrales encendidas estén funcionando simultáneamente a máxima potencia.

  $\sum_jPMax_jv_{jk}\geq D_k+R_k \quad \forall k$

In [3]:
reservas=[]
for i in range(len(demandas)):
  reservas.append(demandas[i]*0.1)#10% de las demandas

J=3 #Tres centrales
periodos=range(len(demandas)) #k=0,...,24
centrales=range(J)
matriz = { (j,k) : 0 for k in periodos for j in centrales if k!=0} #Matriz sin significado, utilizada como un iterable matriz.keys()

### **FUNCIÓN OBJETIVO:**

---

Se busca minimizar los costes de funcionamiento, arranque y parada, restando el beneficio de producción.

Se produce un coste $C_j$ cada vez que se enciende la central $j$, $E_j$ cada vez que se apaga y $A_j$ por cada hora que se mantiene en funcionamiento.
$A_j$ es una cantidad fija, no relacionada con la producción sino con el mero hecho de estar funcionando. Por ejemplo, podría tratarse de los gastos de personal. 

Además, por cada unidad de energía producida en la central $j$ se recibe un beneficio neto $B_j$, donde ya se han descontado implícitamente los costes de producción.

$min\quad Z=\sum_k\sum_j [A_jv_{jk}+C_jy_{jk}+E_jz_{jk}-B_jp_{jk}]$


## **SOLUCIÓN:** 

---



In [4]:
!pip install ortools
from ortools.linear_solver import pywraplp

     |████████████████████████████████| 14.0MB 301kB/s 
     |████████████████████████████████| 1.0MB 34.7MB/s 
     |████████████████████████████████| 133kB 37.4MB/s 
ERROR: tensorflow-metadata 0.26.0 has requirement absl-py<0.11,>=0.9, but you'll have absl-py 0.11.0 which is incompatible.
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4
  Found existing installation: absl-py 0.10.0
    Uninstalling absl-py-0.10.0:
      Successfully uninstalled absl-py-0.10.0


In [5]:
#solver=pywraplp.Solver.CreateSolver('CBC') #no funciona
solver = pywraplp.Solver.CreateSolver('SCIP')
y = { (j,k) : solver.BoolVar('arranque[%i, %i]' % (j, k)) for j in centrales for k in periodos}
z = {(j,k): solver.BoolVar('parada[%i,%i]'%(j,k)) for j in centrales for k in periodos}
v = {(j,k): solver.BoolVar('funciona[%i,%i]'%(j,k)) for j in centrales for k in periodos}
p = {(j,k): solver.NumVar(0,solver.infinity(),name='produccion[%i,%i]'%(j,k)) for j in centrales for k in periodos}

solver.Minimize(solver.Sum(A[j]*v[j,k]-B[j]*p[j,k]+C[j]*y[j,k]+E[j]*z[j,k] for j,k in matriz.keys())) #Objetivo

print('Number of variables =', solver.NumVariables())

Number of variables = 300


In [6]:
#Inicialización: Todas las centrales están apagadas (las variables para k=0 están fijas)
[ solver.Add(y[j,0]==0) for j in centrales]
[ solver.Add(z[j,0]==0) for j in centrales]
[ solver.Add(p[j,0]==0) for j in centrales]
[ solver.Add(v[j,0]==0) for j in centrales]
print('Number of constraints =', solver.NumConstraints())

Number of constraints = 12


In [7]:
#Resto de restricciones
[ solver.Add( Pmin[j]*v[j,k]<=p[j,k] , name='Prod minima')  for j in centrales for k in periodos]
[ solver.Add( p[j,k]<=PMax[j]*v[j,k] , name='Prod maxima')  for j in centrales for k in periodos]
[ solver.Add( p[j,k+1]-p[j,k]<=Subida[j], name='Subida') for j in centrales for k in periodos if k!=max(periodos)]
[ solver.Add( p[j,k]-p[j,k+1]<=Bajada[j], name='Bajada') for j in centrales for k in periodos if k!=max(periodos)]
[ solver.Add( y[j,k]-z[j,k]==v[j,k]-v[j,k-1], name='Estado') for j in centrales for k in periodos if k!=0]
[ solver.Add(solver.Sum(p[j,k] for j in centrales)==demandas[k], name='Demanda') for k in periodos ]
[ solver.Add(solver.Sum(PMax[j]*v[j,k] for j in centrales)>=demandas[k]+reservas[k]) for k in periodos]
print('Number of constraints =', solver.NumConstraints())

Number of constraints = 428


In [8]:
#Solución
solver.Solve()

0

## **RESULTADO:** 

---



In [9]:
result_status = solver.Solve()
print('Infactible? ', result_status== pywraplp.Solver.INFEASIBLE)
print('Optimo? ',result_status== pywraplp.Solver.OPTIMAL)
print( solver.WallTime()/1000, " seconds")
print('Coste = %f' % solver.Objective().Value())
enciende=[]
apaga=[]
produccion=[]
for (cent,per) in matriz.keys():
            if y[cent,per].solution_value() > 0.01 :
              enciende.append((cent,per))
            if z[cent,per].solution_value() > 0.01 :
              apaga.append((cent,per))

for per in periodos:
  produccion.append((per,p[0,per].solution_value(),p[1,per].solution_value(),p[2,per].solution_value()))
   # print("xij=",xij)
print('Arranques: %s' % sorted(enciende,key=lambda x: x[0]))
print('Paradas: %s' % sorted(apaga,key=lambda x: x[0]))

from tabulate import tabulate
print(tabulate(produccion, headers=['Periodo','Nuclear', 'Eólica','Térmica']))

Infactible?  False
Optimo?  True
19.083  seconds
Coste = -2052.940800
Arranques: [(0, 1), (1, 2), (1, 10), (2, 1)]
Paradas: [(1, 4), (2, 2)]
  Periodo    Nuclear    Eólica    Térmica
---------  ---------  --------  ---------
        0      0         0              0
        1     22.772     0             13
        2     25.85      7              0
        3     20         9.316          0
        4     26.999     0              0
        5     25.086     0              0
        6     24.319     0              0
        7     23.627     0              0
        8     23.419     0              0
        9     24.095     0              0
       10     20.184     7              0
       11     20        11.694          0
       12     20        13.905          0
       13     20        15.022          0
       14     20        15.481          0
       15     20        15.904          0
       16     20        14.118          0
       17     20        14.982          0
       18     20   

**Comentarios sobre el resultado**

---



En el primer periodo se encienden la central nuclear y la térmica (con su producción mínima). Esto se debe a que la demanda es tan alta que la rampa de subida de carga máxima de la central eólica (7) es demasiado baja como para completar la demanda junto con la nuclear. 

Para el segundo periodo, la demanda ha bajado y puede apagarse la central térmica, sustituyéndola por la eólica con el máximo permitido por su rampa de subida. 

En el tercer periodo se reduce la producción nuclear al mínimo, explotando las subvenciones por energía eólica.

En los periodos 4-9 la demanda es demasiado baja como para mantener la central nuclear y la eólica encendidas, ni siquiera con su producción mínima. Sin embargo es demasiado alta como para que la central eólica pueda hacerse cargo en solitario. Por lo tanto, se deja encendida sólo la nuclear.

En el periodo 10 la demanda sube lo suficiente como para encender de nuevo la eólica, que aprovecha toda su rampa de subida. La nuclear satisface el resto.

A partir del periodo 11 se mantiene la energía nuclear en producción mínima y se cubre el resto con energía eólica, maximizando las subvenciones.

Al final, tenemos un beneficio de 2052.9408 euros.


**REFERENCIAS:**

---

- Idea del problema (modificado): *Formulación y resolución de modelos de programación matemática en ingeniería y ciencia.* E. Castillo, A.J. Conejo, P. Pedregal, R.García y N. Alguacil. 2002.

- Demandas eléctricas: https://demanda.ree.es/visiona/peninsula/demanda/total, fecha indicada.

